# Package

In [2]:
import pandas as pd
import numpy as np
#from sklearn.neighbors import NearestNeighbors
from numpy import array
from numpy.linalg import norm

# Functions


In [5]:
def movieId_by_urserId(userid):
    """
    Input
    -----
        userid:int
    Return
    -----
    a list of movie id that rated by given user"""
    return list(rating[rating['userId'] == userid]['movieId'])

# Data

In [6]:
#result = pyreadr.read_r('../output/mat_fac.RData') 
#p = pd.read_csv('../output/p.csv')
#rmse = pd.read_csv('../data/rmse.csv')
q = pd.read_csv('../output/q.csv')
rating = pd.read_csv('../data/ml-latest-small/ratings.csv')

In [10]:
q.head(2)#movie

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,1,0.565178,0.717229,0.778567,0.625927,0.842650,-0.002214,0.623139,0.468605,0.365189,...,0.225377,0.058500,-0.336207,-0.634299,-0.611499,-0.944203,-0.860104,-0.845497,-0.395947,0.430513
1,2,-0.638644,-0.751574,-0.901851,-0.026124,-0.749025,-0.669877,-0.226928,-0.340599,-1.100166,...,-0.369489,-0.021417,0.220290,0.478876,-1.077273,0.382798,-0.213710,-0.816581,0.230590,-0.584523


In [15]:
movie_index = list(q.columns[1:]) #list of all movie index,str
movie_index_int = [int(val) for val in movie_index] #int form of movie_index
userId = set(rating['userId'])


# Vj_index = ['1'] # list of given movie index
# movie_index_sub = [item for item in movie_index if item not in Vj_index] #list of movie indexs except the comparied one

In [ ]:
Vj_index = ['1'] # list of given movie index
movie_index_sub = [item for item in movie_index if item not in Vj_index] #list of movie indexs except the comparied one

In [129]:
l_latent_movie = []
for i in movie_index_sub:
   # print(i,type(i)) 2，str
    numerator =array([val
                  for sublist in np.asarray(q[[Vj_index[0]]]).tolist()
                  for val in sublist]).dot(array([val
                  for sublist in np.asarray(q[[i]]).tolist()
                  for val in sublist]))
    
    cosin_s = numerator/(norm([val
                  for sublist in np.asarray(q[[Vj_index[0]]]).tolist()
                  for val in sublist])*norm([val
                  for sublist in np.asarray(q[[i]]).tolist()
                  for val in sublist]))
    l_latent_movie.append(cosin_s)
    
index_min = np.argmin(l_latent_movie) #find the index of min value

# if int(Vj_index[0]) > index_min: 
#     index_similar = index_min
# else:
#     index_similar = index_min+1 #+1 would be the real index of that similar movie
    
# print(index_similar)

5125


# scrachwork

In [54]:
#q.as_matrix(columns=q.columns[1:2]).dot(q.as_matrix(columns=q.columns[2:3]))

In [123]:
l_latent_movie = []
movie_index = q.columns[1:]
# for i in range(len(q.columns[1:])):
#     print(array([val
#                   for sublist in np.asarray(q[[movie_index[i+1]]]).tolist()
#                   for val in sublist]).dot(array([val
#                   for sublist in np.asarray(q[[movie_index[i+2]]]).tolist()
#                   for val in sublist])))
#     #j = i+1
#     l_latent_movie.append(array([val
#                   for sublist in np.asarray(q[[movie_index[i+1]]]).tolist()
#                   for val in sublist]).dot(array([val
#                   for sublist in np.asarray(q[[movie_index[i+2]]]).tolist()
#                   for val in sublist])))

In [4]:
rmse.head()

,Unnamed: 0,train_test,par,epoch,RMSE
0,1,Train,"f = 10 , lambda = 0.01",1,0.825492
1,2,Train,"f = 20 , lambda = 0.01",1,0.792257
2,3,Train,"f = 10 , lambda = 0.1",1,0.896950
3,4,Train,"f = 20 , lambda = 0.1",1,0.859586
4,5,Test,"f = 10 , lambda = 0.01",1,1.313171


In [8]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
samples = [[0, 0, 2], [1, 0, 0], [0, 0, 1]]

In [13]:
neigh = NearestNeighbors(2, 0.4,metric='cosine')
neigh.fit(samples)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                 radius=0.4)

In [14]:
#class sklearn.neighbors.NearestNeighbors(n_neighbors=5, radius=1.0, algorithm=’auto’, leaf_size=30,
#metric=’minkowski’, p=2, metric_params=None, n_jobs=None, **kwargs)

In [15]:
neigh.predict([0,0,5])

AttributeError: 'NearestNeighbors' object has no attribute 'predict'